In [6]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

True

# Добавьте пароли и все такое в .env файл

In [7]:
db_credentials = dict(
    host=os.environ.get('POSTGRES_HOST'),
    port=os.environ.get('POSTGRES_PORT'),
    user=os.environ.get('POSTGRES_USER'),
    password=os.environ.get('POSTGRES_PASSWORD'),
    dbname=os.environ.get('POSTGRES_DB')
)

In [9]:
connect_alchemy = "postgresql+psycopg2://%s:%s@%s/%s" % (
    db_credentials['user'],
    db_credentials['password'],
    db_credentials['host'],
    db_credentials['dbname']
)
engine = create_engine(connect_alchemy)

In [10]:
csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Species']
iris =  pd.read_csv(csv_url, names = col_names)
iris

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [11]:
with engine.connect() as conn:
    sql = 'SELECT * FROM iris_test'
    iris.to_sql('iris_test', con=conn, if_exists='replace', schema='public', index=False)
    df = pd.read_sql_query(sql, conn)
df

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


# Не создаем лишних таблиц, ресурсы конечны!

Или используем contextmanager или не забываем потом закрывать соединение!

In [13]:
with psycopg2.connect(**db_credentials) as conn:
    with conn.cursor() as cur:
        sql = 'DELETE FROM iris_test  where "Species" = \'Iris-setosa\';'
        cur.execute(sql)

with engine.connect() as conn:
    sql = 'SELECT * FROM iris_test'
    df = pd.read_sql_query(sql, conn)
df

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,7.0,3.2,4.7,1.4,Iris-versicolor
1,6.4,3.2,4.5,1.5,Iris-versicolor
2,6.9,3.1,4.9,1.5,Iris-versicolor
3,5.5,2.3,4.0,1.3,Iris-versicolor
4,6.5,2.8,4.6,1.5,Iris-versicolor
...,...,...,...,...,...
95,6.7,3.0,5.2,2.3,Iris-virginica
96,6.3,2.5,5.0,1.9,Iris-virginica
97,6.5,3.0,5.2,2.0,Iris-virginica
98,6.2,3.4,5.4,2.3,Iris-virginica


In [15]:
with psycopg2.connect(**db_credentials) as conn:
    with conn.cursor() as cur:
        sql = 'DROP TABLE IF EXISTS iris_test'
        cur.execute(sql)

# Очень важно закрыть connection, у нас из ограниенное количество

именно поэтому мы создаем engine, а не передаем строку connect_alchemy в DataFrame.to_sql/pd.read_sql_query, т.к. сам он открывает внутри, но не закрывает

In [16]:
engine.dispose()